In [2]:
# import pandas as pd
# from gensim.models import Doc2Vec
# from gensim.models import doc2vec
# from gensim.models.doc2vec import TaggedDocument
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import _pickle as cPickle
from scipy import sparse
from tqdm import tqdm
from sklearn import utils
import numpy as np

In [3]:
df = pd.read_csv("data/cross_label_norm.csv")

for c in ["title_processed", "body_processed"]:
    df[c] = df[c].astype(str)
    
train = df[df.test_tag == 0]
test = df[df.test_tag == 1]

**Note: please create a "vectors" directory inside the data directory**

In [4]:
def save_tfidf_vectors(ngram_range, title_max_features, body_max_features, column_postfix):
    
    tfidf_vectorizer_title = TfidfVectorizer(
        stop_words='english',
        sublinear_tf=True,
        strip_accents='unicode',
        analyzer='word',
        token_pattern=r'\w{2,}',  #vectorize 2-character words or more
        ngram_range=ngram_range,
        max_features=title_max_features)

    X_train_title_tfidf = tfidf_vectorizer_title.fit_transform(train[f"title_{column_postfix}"])
    X_test_title_tfidf = tfidf_vectorizer_title.transform(test[f"title_{column_postfix}"])

    tfidf_vectorizer_body = TfidfVectorizer(
        stop_words='english',
        sublinear_tf=True,
        strip_accents='unicode',
        analyzer='word',
        token_pattern=r'\w{2,}',  #vectorize 2-character words or more
        ngram_range=ngram_range,
        max_features=body_max_features)

    X_train_body_tfidf = tfidf_vectorizer_body.fit_transform(train[f"body_{column_postfix}"])
    X_test_body_tfidf = tfidf_vectorizer_body.transform(test[f"body_{column_postfix}"])

    column_postfix = column_postfix.replace("_","-")
    with open(f"data/vectors/tfidf_{column_postfix}_train_title", 'ab') as f:
        cPickle.dump(X_train_title_tfidf, f)
    with open(f"data/vectors/tfidf_{column_postfix}_train_body", 'ab') as f:
        cPickle.dump(X_train_body_tfidf, f)
    with open(f"data/vectors/tfidf_{column_postfix}_test_title", 'ab') as f:
        cPickle.dump(X_test_title_tfidf, f)   
    with open(f"data/vectors/tfidf_{column_postfix}_test_body", 'ab') as f:
        cPickle.dump(X_test_body_tfidf, f)                                         

In [5]:
save_tfidf_vectors(ngram_range = (1,2), title_max_features = 10000, body_max_features = 20000, column_postfix = "processed")

In [ ]:
print("tfidf finished")

In [ ]:
# def label_sentences(corpus, label_type):
#     """
#     Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
#     We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
#     a dummy index of the post.
#     """
#     labeled = []
#     for i, v in enumerate(corpus):
#         label = label_type + '_' + str(i)
#         labeled.append(doc2vec.TaggedDocument(str(v).split(), [label]))
#     return labeled

# def get_vectors(model, corpus_size, vectors_size, vectors_type):
#     """
#     Get vectors from trained doc2vec model
#     :param doc2vec_model: Trained Doc2Vec model
#     :param corpus_size: Size of the data
#     :param vectors_size: Size of the embedding vectors
#     :param vectors_type: Training or Testing vectors
#     :return: list of vectors
#     """
#     vectors = np.zeros((corpus_size, vectors_size))
#     for i in range(0, corpus_size):
#         prefix = vectors_type + '_' + str(i)
#         vectors[i] = model.docvecs[prefix]
#     return vectors

# def save_d2v_vectors(min_count, title_features_no, body_features_no, column_postfix):
#     X_train_title = label_sentences(train[f"title_{column_postfix}"], 'Train')
#     X_test_title = label_sentences(test[f"title_{column_postfix}"], 'Test')

#     X_train_body = label_sentences(train[f"body_{column_postfix}"], 'Train')
#     X_test_body = label_sentences(test[f"body_{column_postfix}"], 'Test')

#     all_data_title = X_train_title + X_test_title
#     all_data_body = X_train_body + X_test_body

#     model_dbow_title = Doc2Vec(dm=0, vector_size=title_features_no, negative=5, min_count=min_count, alpha=0.065, min_alpha=0.065)
#     model_dbow_title.build_vocab([x for x in tqdm(all_data_title)])
#     for epoch in range(30):
#         model_dbow_title.train(utils.shuffle([x for x in tqdm(all_data_title)]), total_examples=len(all_data_title), epochs=1)
#         model_dbow_title.alpha -= 0.002
#         model_dbow_title.min_alpha = model_dbow_title.alpha
#     train_title_vectors_dbow = get_vectors(model_dbow_title, len(X_train_title), title_features_no, 'Train')
#     test_title_vectors_dbow = get_vectors(model_dbow_title, len(X_test_title), title_features_no, 'Test')

#     model_dbow_body = Doc2Vec(dm=0, vector_size=body_features_no, negative=5, min_count=min_count, alpha=0.065, min_alpha=0.065)
#     model_dbow_body.build_vocab([x for x in tqdm(all_data_body)])
#     for epoch in range(30):
#         model_dbow_body.train(utils.shuffle([x for x in tqdm(all_data_body)]), total_examples=len(all_data_body), epochs=1)
#         model_dbow_body.alpha -= 0.002
#         model_dbow_body.min_alpha = model_dbow_body.alpha
#     train_body_vectors_dbow = get_vectors(model_dbow_body, len(X_train_body), body_features_no, 'Train')
#     test_body_vectors_dbow = get_vectors(model_dbow_body, len(X_test_body), body_features_no, 'Test')
    
#     column_postfix = column_postfix.replace("_","-")
#     with open(f"data/vectors/d2v_{title_features_no}-{body_features_no}_{column_postfix}_train_title", 'ab') as f:
#         cPickle.dump(train_title_vectors_dbow, f)
#     with open(f"data/vectors/d2v_{title_features_no}-{body_features_no}_{column_postfix}_train_body", 'ab') as f:
#         cPickle.dump(train_body_vectors_dbow, f)
#     with open(f"data/vectors/d2v_{title_features_no}-{body_features_no}_{column_postfix}_test_title", 'ab') as f:
#         cPickle.dump(test_title_vectors_dbow, f)   
#     with open(f"data/vectors/d2v_{title_features_no}-{body_features_no}_{column_postfix}_test_body", 'ab') as f:
#         cPickle.dump(test_body_vectors_dbow, f)    

In [ ]:
# save_d2v_vectors(min_count = 2, title_features_no = 500, body_features_no = 500, column_postfix = "proc_lem")